# Exchangeability Martingale Model for Time Series Anomaly Detection
**Harvard University**<br>
**Fall 2016**<br>
**Instructors: W. Pan, P. Protopapas, K. Rader**<br>

<br>

**Dennis Milechin, Ivan Sinyagin, Hany Bassily**

<br>
## Production Code

ver 1.0 12/14/2016 Bassily H., Sinyagin I. and Milechin D.



<br>

## Instructions:

<br>
This code calculate the exchangeabiliy martingale, the change detection difference martingale and the smoothed changed detection difference martingale.

It performs the martingale computation based on three methods, the power martingale with predefined epsilon value, the mix power martingale and the plugin martingale. The user can choose which method to be used to calculate the three model outputs.

For the strangeness measure, the user can choose between 3 methods if the data is single vararaible. The three options are the distance, the standard deviation or the jump rate. If the data is multivariate, only the distance will be consider regardless what selection the user has chosen

### Input:

The code accepts as input two Numeric only csv files. The first includes the data that will be used as training and the second is the test data

### Calculation Method:

The calculation follows the following steps:
 - Data preprocessing where two procedures are available where the user can choose either or both. The first is a normalization procedure where both datasets are normalized by their respective standard deviation
 
 - The second preprocessing is the application of a time lag diference filter where the user can specify the amount of lag to be applied for the difference
 
 - The preprocessed data is then admitted to the martingale calculation algorithm implemented as per Fedorova et al., 2012. The algorithm starts by the calculation of the strangeness measure according to the user selection then the p-value computation and the martingale evaluation according to which martingale method the user selected.
 
 - Each martingale type has its own seperate function
 
 - The outcome of this step is the logarithmic martingale which is subsequently used in calculating the changedetction measure based on the difference between two consecutive martingale
 
 - A smoothing function is then applied to the resulting change detection measure which applies an exponential filter to better use the result for thresholding and persistence evaluation if needed
 
 - For the mix martingale, a numeric integration algorithm is used. For which, discrete values of the epsilon domain should be specified. For this purpose a "resolution" term is introduced and specified by the user to define how fine the epsilon domain can be discretesized.
 
### Options:

The user has to define the following:

 - A full pathh to the training and test data files
 - Selection whether the data to be normalized or not
 - Selection if a lag difference filter should be applied or not and the desired lag
 - The selection of the starngeness function (wil be overriden id the data is multivariate)
 - The type of martingale function
 - The value of epsilon to be used with the predefined epsilon power martingale method
 - The resolution for the integartion of the mixed martingale
 - The smootheness factor
 
### Output

The output of the program is a csv file which includes three columns:

 - The calculated martingale
 - The change detection difference value
 - The smoothed changed detection difference value
 
### Directions:

 - First step clean the data and prepare to csv files. One includes the training set and the second the test set
 - Open the ipython notebook and fill the options sections with all required detail. Please adhere to the syntax indicated for each option
 - run the model and the output file will begenerat

### Import Libraries

<br>

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import time
from scipy.integrate import simps

from inspect import getmembers

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors

from scipy import stats
from scipy.stats import gaussian_kde as PDF

import collections
from matplotlib import rcParams
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

**---**

## 1. Options

<br>

In [2]:
# Options to be specified by the user:
# ------------------------------------

'''This section is used to specify all the entries required to run the model:

        1- Full path of the training dataset
        2- Full path of the test dataset
        3- The required preprocessing (Normalization - Difference)
        4- The type of strangeness function needed (select between three types)
        5- The type of martingale required (Power - Mix - Plugin)
        6- The value of Epsilon to be used for the Power Martingale with predefined Epsilon
        7- The smoothing power for the change detection
        8- The resolution for the mix martingale integration which is the number of slices the interval of 
           [0,1] will be divided to - Vovk et al., 2003 '''

# Training data file
file_train = 'datasets/4205872016.csv'

# Test data file
file_test = 'datasets/4205882016.csv'

# Normalization required [y/n] ?
normal = 'y'

# Difference required (enter required delay or 'No' if no difference is needed ?
dif = 15

# Type of strangeness function ( Distance = 1, Standard Deviation = 2 , Jump rate = 3)
strg = 3

# Type of Martingale to be calculated:
#         Predefined Epsilon = 'eps'
#         Mixed Martingale   = 'mix' 
#         Plugin Martingale  = 'plg'
mart = 'eps'

# the value of Epsilon to be used for the Power Martingale with predefined Epsilon [0,1]
eps = 0.3

# Mixed Martingale Integration Resolution (i.e. the number of slices the interval [0,1] will be divided to to 
# integrate over all Epsilon values - Vovk et al., 2003)
res = 100

# Smoothing power of the filer [0,1]
a = 0.008

## 2. General Functions

<br>
### 2.1 Power Martingale with Predefined Epsilon Value

<br>

In [3]:
# Function to calculate Power Martingale - Single Epsilon:
# -------------------------------------------------------
# power_martingale function
# Description: Calculates Power Martingale for outlier detection
# Input:
#       train - numpy matrix containing the training data set
#       test - numpy matrix containing the testing data set
#       eps - epsilon value
#       alpha_flag - 1,2,3
# Return:
#       datpower_mart - Power Martingale Value
#       diff_arr - Power Martingale Value Difference
#
def power_martingale(train , test , eps , alpha_flag):
 
    # data mean
    mean = np.average(train , axis = 0)
    
    # mean vector norm
    mean_norm = np.linalg.norm(mean)

    # strangenes reference for distance
    ref_alpha = np.linalg.norm(train.std(axis = 0)) + mean_norm

    # Data shape
    L = test.shape[0]
    col = test.shape[1]
    
    # Only the distance strangeness is available for multivariate data
    if col>1:
        alpha_flag = 1
    
    # alpha
    if alpha_flag == 1:
        alpha = np.linalg.norm(test , axis = 1) / ref_alpha
        
    elif alpha_flag == 2:
        alpha = np.absolute(test - np.average(train)) /  train.std(axis=0)
        
    else:
        alpha = alpha_calc(train , test)

    # p-value(randomised)
    p_value = np.ones(alpha.shape)

    # iterrate for p-value calculation
    for i in range(1,L):
    
        # end value
        end = alpha[i]
    
        # alpha subset
        alpha_sub = alpha[:i + 1]

        # number of elements
        n = i + 1   
    
        # p-values calculation
        np.random.seed(200)
        p_value[i] = ( float(np.sum(alpha_sub > end)) + np.random.uniform() * np.sum(alpha_sub == end ) ) / float(n)
    
        # to avoid log zero
        p_value[p_value==0] = 0.0000000001
    
    # initiate Power Martingale
    power_mart = np.zeros(alpha.shape)

    # dummy multiplier
    M = 0.
    
    # initiate difference measure
    diff_arr = np.zeros((L,))

    # iterrate for power martingale calculation
    for i in range(L):
        
        # increment
        delta = eps * (p_value[i]) ** (eps - 1.)
        
        # Calculate martingale difference
        diff_arr[i] = np.absolute(np.log(delta))
        
        # Update power martingal
        M += np.log(delta)  
        
        # Array
        power_mart[i] = M
        
    return power_mart , diff_arr

### 2.2 Mixed Power Martingale

<br>

In [4]:
# Function to calculate Mix Power Martingale:
# -------------------------------------------
# power_martingale_mix function
# Description: Calculates Power Martingale for outlier detection with mixture
# Input:
#       train - numpy matrix containing the training data set
#       test - numpy matrix containing the testing data set
#       res - Resolution of the uniform distribution for integrating epsilon values.
#       alpha_flag - 1,2,3
# Return:
#       datpower_mart - Power Martingale Value
#       diff_arr - Power Martingale Value Difference
#
def power_martingale_mix(train, test , res, alpha_flag):
    
    # epsilon array
    eps = np.linspace(0.001, 0.999, res)
    
    # data mean
    mean = np.average(train , axis = 0)
    
    # mean vector norm
    mean_norm = np.linalg.norm(mean)

    # strangenes reference for distance
    ref_alpha = np.linalg.norm(train.std(axis = 0)) + mean_norm
    

    # Data shape
    L = test.shape[0]
    col = test.shape[1]
    
    # Only the distance strangeness is available for multivariate data
    if col>1:
        alpha_flag = 1

    # alpha
    if alpha_flag == 1:
        alpha = np.linalg.norm(test, axis = 1) / ref_alpha
        
    elif alpha_flag == 2:
        alpha = np.absolute(test - np.average(train)) /  train.std(axis=0)
        
    else:
        alpha = alpha_calc(train , test)
    

    # p-value(randomised)
    p_value = np.ones(alpha.shape)

    # iterrate for p-value calculation
    for i in range(1,L):
    
        # end value
        end = alpha[i]
    
        # alpha subset
        alpha_sub = alpha[:i - 1]

        # number of elements
        n = i + 1   
    
        # p-values calculation
        np.random.seed(200)
        p_value[i] = ( float(np.sum(alpha_sub > end)) + np.random.uniform() * np.sum(alpha_sub == end ) ) / float(n)
    
        # to avoid log zero
        p_value[p_value==0] = 0.00000000001
    
    # initiate Power Martingale
    power_mart = np.zeros(alpha.shape)
    
    # initiate difference
    diff_arr = np.zeros(alpha.shape)

    # dummy multiplier
    M = 0.

    # iterrate for power martingale calculation
    for i in range(L):
    
        # Calculate delta Array    
        delta = np.log(eps) + (eps - 1.) * np.log(p_value[i])
        
        # Integrate for mix
        delta_mix = simps(delta , eps)
        
        # update power martingale
        M += delta_mix
        
        # Array
        power_mart[i] = M
        
        # difference
        diff_arr[i] = np.absolute(delta_mix)
        
        
    return power_mart , diff_arr

### 2.3 Plugin Martingale

<br>

In [5]:
# Function to calculate Plugin Martingale:
# ---------------------------------------
# plugin_martingale_o function
# Description: Calculates Power Martingale for outlier detection with a plugin variation
# Input:
#       train - numpy matrix containing the training data set
#       test - Lnumpy matrix containing the testing data set
#       alpha_flag - 1,2,3
# Return:
#       datpower_mart - Power Martingale Value
#       diff_arr - Power Martingale Value Difference
#
def plugin_martingale(train , test , alpha_flag):
    
    # data mean
    mean = np.average(train , axis = 0)
    
    # mean vector norm
    mean_norm = np.linalg.norm(mean)

    # strangenes reference
    ref_alpha = np.linalg.norm(train.std(axis = 0)) + mean_norm
    # ref_alpha = train.std(axis=0)

     # Data shape
    L = test.shape[0]
    col = test.shape[1]
    
    # Only the distance strangeness is available for multivariate data
    if col>1:
        alpha_flag = 1

    # alpha
    if alpha_flag == 1:
        alpha = np.linalg.norm(test , axis = 1) / ref_alpha
        
    elif alpha_flag == 2:
        alpha = np.absolute(test - np.average(train)) /  train.std(axis=0)
        
    else:
        alpha = alpha_calc(train , test)
    
    # p-value(randomised)
    p_value = np.zeros(alpha.shape)

    # iterrate for p-value calculation
    for i in range(1,L):
    
        # end value
        end = alpha[i]
    
        # alpha subset
        alpha_sub = alpha[:i+1]

        # number of elements
        n = i + 1   
    
        # p-values calculation
        p_value[i] = ( float(np.sum(alpha_sub > end)) + np.random.uniform() * np.sum(alpha_sub == end ) ) / float(n)
    
        # to avoid log zero
        p_value[p_value==0] = 0.00000001
        
    # Extended sample
    p_value_neg = - p_value
    p_value_ref = 2.0 - p_value
    
    # initiate Power Martingale
    power_mart = np.zeros(alpha.shape)
    
    # initiate difference
    diff_arr = np.zeros(alpha.shape)

    # dummy multiplier
    M = 0.

    # iterrate for plugin martingale calculation
    for i in range(1,L):
        
        # samples
        s1 = p_value_neg[:i]
        s2 = p_value[:i]
        s3 = p_value_ref[:i]
        
        # concatenation
        s12 = np.concatenate((s1 , s2) , axis = 0)
        sample = np.concatenate((s12 , s3) , axis  = 0)
       
        
        # Estimate pdf
        den = PDF(sample, bw_method='silverman')
        
        # Integral
        A = den.integrate_box_1d(0,1)
        
        # Evaluate
        f =  den.evaluate(p_value[i]) / A
        delta = np.log(f)
        M += delta
        power_mart[i] = M 
        diff_arr[i] = delta
    
    # First Element
    power_mart[0] = power_mart[1]   
    
    
    return power_mart , diff_arr

### 2.4 Jump Rate Strangeness

<br>

In [6]:
# alpha_calc function
# Description: Calculates Alpha Values
# Input:
#       train - numpy matrix containing the training data set
#       test - Lnumpy matrix containing the testing data set
# Return:
#       Alpha Values

def alpha_calc(train , test):
    
    # initiation
    train_d = np.zeros(train.shape)
    test_d = np.zeros(test.shape)
    
    # Calculate refeence jump rate from training
    for i in range(1,train_d.shape[0]):
        train_d[i] = np.absolute(train[i] - train[i-1])
    
    # Calculate test jump rate
    for i in range(1, test_d.shape[0]):
        test_d[i] = np.absolute(test[i] - test[i-1])

    return test_d / np.average(train_d)

### 2.5 Smoothing

<br>

In [7]:
# Exponential Smoother:
# ---------------------
# smooth function
# Description: Exponential Smoother
# Input:
#       y - Data to be smoothed
#       alpha - alpha parameter
# Return:
#       x - smoothed values
def smooth(y , alpha):
    
    # initiation of the filtered signal array
    x = np.zeros(y.shape)
    
    # Data size
    L = y.shape[0]
    
    # intiation
    x[0] = 0.5 * (1. + alpha) * y[0]
    
    # Iterrate for new samples
    for i in range(1,L):
        x[i] = alpha * y[i] + (1. - alpha) * x[i - 1]
        
    return x

## 3. Data Loading and Preprocessing

<br>

In [8]:
# Open both training and test data sets:
# -------------------------------------

# training set 
train_raw = np.loadtxt(file_train, delimiter = ',') 

# test set
test_raw = np.loadtxt(file_test , delimiter = ',')

In [9]:
# Data preprocessing:
# -------------------

# Training data shape
train_len = train_raw.shape[0]     
train_col = train_raw.shape[1]

# Test data shape
test_len = test_raw.shape[0]
test_col = test_raw.shape[1]


# Normaliztion
if (normal== 'y'):
    train_norm = train_raw / train_raw.std(axis=0)
    test_norm  = test_raw / test_raw.std(axis=0)
    
else:
    train_norm = train_raw
    test_norm = test_raw
    

# Difference
if (dif!='No'):
    
    # Initiate arrays
    train = np.zeros(train_norm.shape)
    test  = np.zeros(test_norm.shape)
    
    # Training data
    for i in range (dif, train_len):
        train[i, :] = train_norm[i,:] - train_norm[ i - dif , : ]
        
    # Test data
    for i in range (dif, test_len):
        test[i, :] = test_norm[i,:] - test_norm[ i - dif , : ]    
    
    
else:
    train = train_norm
    test  = test_norm      
    
    

## 4. Calculation

<br>

In [10]:
# Calculate the Martingales and change detection:
# ----------------------------------------------

# Fixed Epsilon
if (mart == 'eps'):
    martingale , cd_diff = power_martingale(train , test , eps , strg)
    
# Mixed Martingale
elif (mart == 'mix'):
    martingale , cd_diff = power_martingale_mix(train, test , res, strg)
    
# Plugin Martingale
else:
    martingale , cd_diff = plugin_martingale(train , test , strg)
    
# Smooth the change detection difference:
cd_smooth = smooth(cd_diff , a)

## 5. Output (.csv file)

<br>

In [11]:
# Construct the output data frame and export to a text file:
# ---------------------------------------------------------

# Reshape arrays
array_mar  = martingale.reshape((test_len,1))
array_diff = cd_diff.reshape((test_len , 1))
array_smooth = cd_smooth.reshape((test_len , 1))

# Concatenate
results_1 = np.concatenate((array_mar , array_diff) , axis  = 1)
results_array = np.concatenate((results_1 , array_smooth) , axis = 1)

# Data FRame and Export
results_df = pd.DataFrame(results_array , columns = ['Martingale' , 'ChangeDetection' , 'SmoothedChangedDetection'])

# Export file
results_df.to_csv('ExchangeabilityMartingale.csv')

---